In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
#from rake_nltk import Rake
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import re

In [4]:
df_trainfile=pd.read_csv("/content/drive/My Drive/Innoplexus/train_F3WbcTw_1.csv")
df_testfile=pd.read_csv("/content/drive/My Drive/Innoplexus/test_tOlRoBf_1.csv")
print(df_trainfile.shape)
print(df_testfile.shape)

(5279, 4)
(2924, 3)


In [0]:
df_trainfile.set_index('unique_hash',inplace=True)
df_testfile.set_index('unique_hash',inplace=True)

In [0]:
df_trainfile["text_drug"]=df_trainfile["text"]



df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: x.lower())

df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\W', ' ', str(x)))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\s+[a-zA-Z]\s+', ' ', x))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\^[a-zA-Z]\s+', ' ', x))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\s+', ' ', x, flags=re.I))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'^b\s+', '', x))

In [7]:
print(df_trainfile.sentiment.value_counts())
count_class_2, count_class_1, count_class_0=df_trainfile.sentiment.value_counts()
print((count_class_2))
print((count_class_1))
print((count_class_0))

2    3825
1     837
0     617
Name: sentiment, dtype: int64
3825
837
617


In [0]:
df_class_0 = df_trainfile[df_trainfile['sentiment'] == 0]
df_class_1 = df_trainfile[df_trainfile['sentiment'] == 1]
df_class_2 = df_trainfile[df_trainfile['sentiment'] == 2]

In [9]:
df_class_0_over = df_class_0.sample(count_class_2, replace=True)
df_class_1_over = df_class_1.sample(count_class_2, replace=True)

print(df_class_0_over.shape)
print(df_class_1_over.shape)
print(df_trainfile.shape)

(3825, 4)
(3825, 4)
(5279, 4)


In [10]:
df_test_over = pd.concat([df_class_0_over, df_class_1_over,df_class_2], axis=0)
print(df_test_over.shape)

(11475, 4)


In [14]:
  import nltk
  nltk.download('punkt')
  nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [15]:
print(df_test_over["sentiment"].value_counts())

lemmer=nltk.stem.WordNetLemmatizer()

def Lem_lemmatize(tokens):
  return [lemmer.lemmatize(token) for token in tokens]

def Lem_tokens(x):
  return Lem_lemmatize(nltk.word_tokenize(x))

clf_tfidf=TfidfVectorizer(lowercase=True,stop_words="english", tokenizer=Lem_tokens)

x_text_train=df_test_over["text_drug"]
x_text_train_trf=clf_tfidf.fit_transform(x_text_train).toarray()

2    3825
1    3825
0    3825
Name: sentiment, dtype: int64


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
y=df_test_over["sentiment"]

X_train,X_test,y_train,y_test=train_test_split(x_text_train_trf,y,test_size=0.4)

In [18]:
clf_rfc=RandomForestClassifier(random_state=10)

#v_estimators=[50,70,100,150]
v_estimators=[100,150,200]
#v_max_depth=[int(x) for x in np.linspace(start=10,stop=200,num=20)]
#v_max_depth=[50,100,150]
v_max_depth=[100,150,200]
#v_min_samples_leaf=[2,3,4,5]


param_grid={"n_estimators":v_estimators,
            "max_depth":v_max_depth}

clf_gridsearch=GridSearchCV(estimator=clf_rfc,
                           param_grid=param_grid,
                           cv=3,
                           verbose=10)

clf_gridsearch.fit(X_train,y_train)
print(clf_gridsearch.best_params_)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] max_depth=100, n_estimators=100 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... max_depth=100, n_estimators=100, score=0.884, total=  34.8s
[CV] max_depth=100, n_estimators=100 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   34.8s remaining:    0.0s


[CV] ..... max_depth=100, n_estimators=100, score=0.894, total=  37.3s
[CV] max_depth=100, n_estimators=100 .................................


...

[CV] ..... max_depth=100, n_estimators=200, score=0.887, total= 1.7min
[CV] max_depth=150, n_estimators=100 .................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  8.8min remaining:    0.0s


[CV] ..... max_depth=150, n_estimators=100, score=0.890, total=  50.8s
[CV] max_depth=150, n_estimators=100 .................................
[CV] ..... max_depth=150, n_estimators=100, score=0.896, total=  34.8s
[CV] max_depth=150, n_estimators=100 .................................
[CV] ..... max_depth=150, n_estimators=100, score=0.889, total=  35.4s
[CV] max_depth=150, n_estimators=150 .................................
[CV] ..... max_depth=150, n_estimators=150, score=0.891, total=  51.2s
[CV] max_depth=150, n_estimators=150 .................................
[CV] ..... max_depth=150, n_estimators=150, score=0.900, total=  51.8s
[CV] max_depth=150, n_estimators=150 .................................
[CV] ..... max_depth=150, n_estimators=150, score=0.894, total=  51.8s
[CV] max_depth=150, n_estimators=200 .................................
[CV] ..... max_depth=150, n_estimators=200, score=0.895, total= 1.1min
[CV] max_depth=150, n_estimators=200 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 25.9min finished


{'max_depth': 150, 'n_estimators': 200}


In [0]:
clf_rfc_f=RandomForestClassifier(random_state=10, n_estimators=200,max_depth=150) 
#-- without min_sample leaf 0.961811076903026
# without drug addition 0.9622244594662654
#0.9597993595429312
clf_rfc_f.fit(X_train,y_train)
y_pred=clf_rfc_f.predict(X_test)

In [20]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
print(f1_score(y_test,y_pred,average="macro"))
print(classification_report(y_test,y_pred))

0.9572996953629351
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1547
           1       0.95      0.95      0.95      1501
           2       0.93      0.94      0.94      1542

    accuracy                           0.96      4590
   macro avg       0.96      0.96      0.96      4590
weighted avg       0.96      0.96      0.96      4590



In [21]:
print(len(y_test))
#print(y_pred)

df_y_test=pd.DataFrame(data=y_test.values,columns=["Actual"])
df_y_test["Predicted"]=y_pred

print(df_y_test["Predicted"].value_counts())
print(df_y_test["Actual"].value_counts())

4590
2    1556
0    1524
1    1510
Name: Predicted, dtype: int64
0    1547
2    1542
1    1501
Name: Actual, dtype: int64
